In [6]:
#import important libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, normalize
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from prettytable import PrettyTable
from sklearn.preprocessing import Normalizer
import math

In [7]:
# Import data from txt format to dataframe
df = pd.read_csv("746Data.txt", delimiter=",",  names=["Peptides", "Result"])
df.head()

,Peptides,Result
0,AAAKFERQ,-1
1,AAAMKRHG,-1
2,AAAMSSAI,-1
3,AAKFERQH,-1
4,AAKFESNF,-1


In [8]:
# Seperate all amino acids
peptides = np.array([[df["Peptides"][i][j] for i in range(df.shape[0])] for j in range(8)])
df.shape

(746, 2)

In [9]:
# Store the seperated amino acids into a dataframe
df2 = pd.DataFrame(peptides.T, columns=list('ABCDEFGH'))
df2.shape
df2.head()

,A,B,C,D,E,F,G,H
0,A,A,A,K,F,E,R,Q
1,A,A,A,M,K,R,H,G
2,A,A,A,M,S,S,A,I
3,A,A,K,F,E,R,Q,H
4,A,A,K,F,E,S,N,F


In [10]:
df = df.assign(**df2)
df.head()

,Peptides,Result,A,B,C,D,E,F,G,H
0,AAAKFERQ,-1,A,A,A,K,F,E,R,Q
1,AAAMKRHG,-1,A,A,A,M,K,R,H,G
2,AAAMSSAI,-1,A,A,A,M,S,S,A,I
3,AAKFERQH,-1,A,A,K,F,E,R,Q,H
4,AAKFESNF,-1,A,A,K,F,E,S,N,F


In [11]:
df = df.drop(columns="Peptides", axis=0)
df.head()

,Result,A,B,C,D,E,F,G,H
0,-1,A,A,A,K,F,E,R,Q
1,-1,A,A,A,M,K,R,H,G
2,-1,A,A,A,M,S,S,A,I
3,-1,A,A,K,F,E,R,Q,H
4,-1,A,A,K,F,E,S,N,F


In [12]:
x = np.array(df.drop(columns="Result", axis=0))
y = np.array(df.Result)
pd.DataFrame(x).head()

,0,1,2,3,4,5,6,7
0,A,A,A,K,F,E,R,Q
1,A,A,A,M,K,R,H,G
2,A,A,A,M,S,S,A,I
3,A,A,K,F,E,R,Q,H
4,A,A,K,F,E,S,N,F


In [13]:
# Encoding Categorical Data
labelencoder_X = LabelEncoder()

for i in range(8):
    x[:, i] = labelencoder_X.fit_transform(x[:, i])
    
onehotencoder = OneHotEncoder(categorical_features = "all", sparse=True)
x = onehotencoder.fit_transform(x).toarray()
y = np.array(df.Result)


In [16]:
pd.DataFrame(x).head()

,0,1,2,3,4,5,6,7,8,9,...,150,151,152,153,154,155,156,157,158,159
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
pd.DataFrame(y).head()

,0
0,-1
1,-1
2,-1
3,-1
4,-1


In [18]:
colums = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

In [19]:
#Using sklearn linear regression model

x_data_train, x_data_test, y_data_train, y_data_test = train_test_split(
        x,y, test_size=0.25,random_state=42)
    
algorithms = {'DecisionTreeClassifier(random_state=0)':'DecisionTreeClassifier', 
              'KNeighborsClassifier(n_neighbors=3)': 'KNN',
              "LogisticRegression()" : "LogisticRegression"
             }
acc_train_test = []
r2_train_test = []
results =  {}
for key, value in algorithms.items():        
        model = eval(key)
        reg = model.fit(x_data_train,y_data_train)            
        y_pred_train =  reg.predict(x_data_train)
        y_pred_test =  reg.predict(x_data_test)

        r2_train_test.append([value, (r2_score(y_data_train,y_pred_train),r2_score(y_data_test,y_pred_test)), 
                          (accuracy_score(y_data_train,y_pred_train), accuracy_score(y_data_test,y_pred_test))])
results.update({"data": r2_train_test})        
x1 = PrettyTable()
x1.field_names = ["Algorithm", "R2Score (Train/Test)", "Accuracy (Train/Test)"]
r2score = results['data']
for val in range(0 , len(r2score)):
    r2_value = str(math.floor(r2score[val][2][0]*100)) + '/' + str(math.floor(r2score[val][2][1]*100))
    acc_value = str(math.floor(r2score[val][1][0]*100)) + '/' + str(math.floor(r2score[val][1][1]*100))
    x1.add_row([r2score[val][0], r2_value, acc_value])
print(x1)
print(classification_report(y_data_train,y_pred_train,target_names=colums))
print(classification_report(y_data_test,y_pred_test, target_names=colums))
print(confusion_matrix(y_data_test, y_pred_test))
print(confusion_matrix(y_data_train, y_pred_train))

+------------------------+----------------------+-----------------------+
|       Algorithm        | R2Score (Train/Test) | Accuracy (Train/Test) |
+------------------------+----------------------+-----------------------+
| DecisionTreeClassifier |        100/87        |         100/48        |
|          KNN           |        90/81         |         63/26         |
|   LogisticRegression   |        97/94         |         89/76         |
+------------------------+----------------------+-----------------------+
             precision    recall  f1-score   support

          A       1.00      0.95      0.97       257
          B       0.96      1.00      0.98       302

avg / total       0.97      0.97      0.97       559

             precision    recall  f1-score   support

          A       0.99      0.89      0.93        87
          B       0.91      0.99      0.95       100

avg / total       0.94      0.94      0.94       187

[[77 10]
 [ 1 99]]
[[243  14]
 [  1 301]]
